In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("../code/")

In [2]:
from common_functions import *

In [3]:
import pickle

In [4]:
out_overall = pickle.load(open("metadata/all_regions_years.pkl"))

In [5]:
df = out_overall[2014]['Austin']

In [6]:
num_homes = df.shape[0]
appliance = 'fridge'
valid_homes = valid_homes_data['Austin'][appliance]
appliances = ['aggregate','hvac','fridge']
months = range(1, 13)

In [7]:
df_a = df.ix[valid_homes]
df_a.shape

(339, 137)

In [8]:
df_aggregate = df_a[['aggregate_%d' %month for month in range(1, 13)]]

In [9]:
valid_homes_filtered = df_aggregate[~((df_aggregate>2000).sum(axis=1)>3)].index

In [10]:
to_use_homes = df_a.ix[valid_homes_filtered].query('full_agg_available==1 & area>0.0 & num_occupants>0.0 & house_num_rooms>0.0').index

In [11]:
def normalize(x, maximum, minimum):
    return (x-minimum)*1.0/(maximum-minimum)

In [12]:
maximum = df_aggregate.ix[to_use_homes].max().max()
minimum = 0.0

In [13]:
if appliance=="hvac":
    months = 6
    start, stop=5, 11
else:
    months = 12
    start, stop = 1, 13
tensor = np.zeros((len(to_use_homes), 2, months))
for i, home in enumerate(to_use_homes):
    home_row = df.ix[home]
    for j, appliance in enumerate(['aggregate','hvac']):
        for k, month in enumerate(range(start, stop)):
            tensor[i, j, k] = normalize(home_row['%s_%d' %(appliance, month)], maximum, minimum)

In [14]:
tensor.shape

(57, 2, 12)

In [17]:
np.sum(np.isnan(tensor))

17

In [18]:
def un_normalize(x, maximum, minimum):
    return (maximum-minimum)*x + minimum

In [19]:
import tensorly

In [20]:
from tensorly.decomposition import parafac
factors = parafac(tensor, rank=2)

ValueError: array must not contain infs or NaNs

In [49]:
out = {}
dfs = {}
import time, datetime
for h in range(1, 3):
    print "#"*20
    print "Iteration "+str(h) + " Started at:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print "#"*20
    
    
    out[h] = {}

    for i, home in enumerate(to_use_homes[:]):
        print i, home
        print "*"*20
        tensor_copy = tensor.copy()
        j = 1# For appliance
        for k, month in enumerate(range(start, stop)):
            tensor_copy[i, j, k] =np.NaN
        H, A, T = tensor_fact(tensor_copy, h, h)
        pred_tensor = np.tensordot(np.tensordot(H, A, axes=1), T, axes=1)
        out[h][home] = un_normalize(pred_tensor[i, 1, :], maximum, minimum)
    out_df = pd.DataFrame(out[h]).T
    out_df.columns = [['%s_%d' %(appliance, month) for month in range(start, stop)]]
    dfs[h] = out_df
        
        
        

####################
Iteration 1 Started at: 2017-03-30 17:06:39
####################
0 26
********************
1 59
********************
2 94
********************
3 135
********************
4 222
********************
5 370
********************
6 410
********************
7 545
********************
8 624
********************
9 661
********************
10 871
********************
11 936
********************


KeyboardInterrupt: 

In [18]:
dfs[1]

,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10
26,339.762024,484.516479,574.866638,655.803467,468.965149,348.447845
59,312.586945,445.750854,528.453979,603.438110,431.471161,320.711578
94,123.935524,176.856735,209.667831,239.521835,171.209885,127.178085
135,115.780464,165.221664,195.876282,223.772171,159.955933,118.819443
222,577.118835,822.189209,975.021545,1114.677490,796.300903,591.934814
370,388.712708,554.568359,658.829895,752.583252,538.095825,399.829742
410,185.840332,265.048157,314.210663,358.932312,256.626190,190.715363
545,419.576843,597.624390,708.241028,809.530518,578.751404,430.381775
624,173.316193,247.230453,293.005615,334.794830,239.337784,177.831757
661,374.267639,533.210327,631.724426,721.607422,516.139282,383.682648


In [51]:
pickle.dump(out_df, open("pred_hvac.pkl",'w'))

In [52]:
pickle.load(open("pred_hvac.pkl",'r'))

,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10
26,353.482422,494.207092,576.641541,684.187866,488.900177,351.275574
59,365.830017,529.267212,627.339294,670.972046,402.811554,239.994614
94,122.462845,167.712402,223.440323,216.070816,177.376129,89.880463
135,113.682365,147.974823,214.734543,232.501251,178.639236,105.639671
222,590.162292,832.919189,1058.103638,1191.932251,805.907288,580.461365
370,548.287415,752.610779,815.018188,850.583191,635.651794,422.591248
410,164.653137,248.201752,323.515106,388.217896,230.968384,128.062149
545,284.181122,399.013184,476.740906,547.888672,388.357239,290.874756
624,221.164673,300.367096,357.160431,404.843628,293.693024,214.713242
661,377.485046,626.267456,718.781067,893.841309,555.517456,399.328308


In [21]:
gtf, prf, rms = compute_rmse_fraction(appliance, dfs[2])

In [25]:
dfs[1]

,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10
26,339.762024,484.516479,574.866638,655.803467,468.965149,348.447845
59,312.586945,445.750854,528.453979,603.438110,431.471161,320.711578
94,123.935524,176.856735,209.667831,239.521835,171.209885,127.178085
135,115.780464,165.221664,195.876282,223.772171,159.955933,118.819443
222,577.118835,822.189209,975.021545,1114.677490,796.300903,591.934814
370,388.712708,554.568359,658.829895,752.583252,538.095825,399.829742
410,185.840332,265.048157,314.210663,358.932312,256.626190,190.715363
545,419.576843,597.624390,708.241028,809.530518,578.751404,430.381775
624,173.316193,247.230453,293.005615,334.794830,239.337784,177.831757
661,374.267639,533.210327,631.724426,721.607422,516.139282,383.682648


In [33]:
pred_df = dfs[2]

In [34]:
import pickle
import pandas as pd
import numpy as np
import itertools
from features import feature_map
from degree_days import dds, dd_keys
import os
from sklearn.metrics import mean_squared_error

upper_limit = {
'hvac': 40,
'fridge': 10,
'wm': 1
}

### FIX This
data_path = os.path.expanduser("~/git/scalable-nilm/create_dataset/metadata/all_regions_years.pkl")

out_overall = pickle.load(open(data_path, 'r'))

year = 2014
train_regions = [region]
train_fraction_dict = {'SanDiego': 1.0, 'Austin': 0.0, 'Boulder': 0.0}
test_region = region
test_home = pred_df.index[0]
feature_list = ['energy']
df = out_overall[2014]['Austin']
gt_df = df[pred_df.columns].ix[pred_df.index]
if appliance == "hvac":
    start, stop = 5, 11
else:
    start, stop = 1, 13
aggregate_df = df.ix[pred_df.index][['aggregate_%d' % month for month in range(start, stop)]]

aggregate_df.columns = gt_df.columns
rows, cols = np.where((aggregate_df < 100))
for r, c in zip(rows, cols):
    r_i, c_i = aggregate_df.index[r], aggregate_df.columns[c]
    aggregate_df.loc[r_i, c_i] = np.NaN

gt_fraction = gt_df.div(aggregate_df) * 100
pred_fraction = pred_df.div(aggregate_df) * 100

gt_fraction_dropna = gt_fraction.unstack().dropna()
pred_fraction_dropna = pred_fraction.unstack().dropna()
pred_fraction_dropna = pred_fraction_dropna.ix[gt_fraction_dropna.index]

rms = np.sqrt(mean_squared_error(gt_fraction_dropna, pred_fraction_dropna))

In [35]:
gt_fraction.describe()

,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10
count,73.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,21.799390,38.527067,43.711273,47.203832,38.336314,25.798282
std,10.112824,11.817828,11.399966,11.446348,10.069190,9.823136
min,0.058835,0.365887,17.892239,19.840344,18.330842,3.159448
25%,14.667286,31.622447,35.139171,38.658659,31.170312,18.340133
50%,23.100827,40.133906,44.741894,48.691038,38.457611,27.107173
75%,28.469579,46.819437,52.921596,56.562197,44.533168,31.507407
max,50.205361,65.285636,70.278192,72.253538,64.411425,49.326522


In [36]:
pred_fraction.describe()

,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10
count,74.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,37.600465,43.549870,45.077591,45.310244,41.727432,38.168484
std,19.283143,14.195603,13.090199,11.619996,12.124201,12.488147
min,-0.043308,18.396687,17.387252,22.965690,14.178800,4.102546
25%,28.900279,36.452025,38.924018,38.112931,35.564866,30.570984
50%,34.683498,40.527463,43.081245,43.015406,40.160036,36.797717
75%,41.278270,46.946586,48.633098,49.747927,44.966401,43.937430
max,137.115864,93.701615,93.142967,83.863121,84.442802,82.944202


In [37]:
rms

19.380868616227314

In [22]:
import tensorflow as tf

In [23]:
M, h=10, 3
W = tf.Variable(tf.random_uniform([M, h], minval=0.1, maxval=1.0))
